In [14]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [15]:
import argparse
import torch
from torch import nn
import matplotlib.pyplot as plt
from stu import STU
import time
import random

In [18]:
from lds import LDS

In [8]:
layer_i = 0
state_dim = 768
batch_size = 10
epochs = 4000
seq_len = 1024
kx = 10
lr = 0.0001

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the layer i weights
stu_layer_full = torch.load(f"../stu_layers/stu_layer_{layer_i}_500m_param_full.pt", map_location=torch.device('cpu'))
stu_layer_full.to(device)

# Initialize LDS model
lds = LDS(state_dim, 768, 768, kx).to(device)
optimizer = torch.optim.Adam(lds.parameters(), lr=lr)

# Training
lds_loss_values = []

best_loss = float('inf')

/var/folders/42/5cczn_jx4454n6fvhcg5qbjw0000gn/T/ipykernel_83753/3556665184.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stu_layer_full = torch.load(f"../stu_layers/

In [9]:
for epoch in range(epochs):
    inputs = torch.randn(batch_size, seq_len, 768).to(device).to(torch.bfloat16)
    stu_outputs = stu_layer_full(inputs).to(device)

    optimizer.zero_grad()
    loss = lds.compute_loss(inputs.to(stu_outputs.dtype), stu_outputs)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(lds.parameters(), max_norm=1)
    lds_loss_values.append(loss.item())
    optimizer.step()

    with torch.no_grad():
        lds.A.data.clamp_(max=1, min=-1)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.1116056740283966
Epoch 10, Loss: 0.0946127325296402
Epoch 20, Loss: 0.08109938353300095
Epoch 30, Loss: 0.07006219029426575
Epoch 40, Loss: 0.060927845537662506
Epoch 50, Loss: 0.052930887788534164
Epoch 60, Loss: 0.045767590403556824
Epoch 70, Loss: 0.0393131747841835
Epoch 80, Loss: 0.03388487920165062
Epoch 90, Loss: 0.029202591627836227
Epoch 100, Loss: 0.024985160678625107


KeyboardInterrupt: 